## III - Неопределенность оптимального портфеля. Оптимизация CVaR. 

Целью работы является оценка неопределенности оптимального портфеля для нормального многомерного распределения и двух способов вычисления оптимального портфеля (оптимизация в модели Марковица и оптимизация CVaR)

### Подготовка модели

#### Загружаем тикеры с фоднового рынка NASDAQ

In [107]:
import pandas as pd

DATA_PATH    = './downloader-data'
TICKERS_PATH = './tickers'

def read_tickers(stock_markets) -> pd.DataFrame:
    '''Returns pandas dataframe containing all tickers for the @stock_markets '''
    ticker_files = [f'{TICKERS_PATH}/{sm}.csv' for sm in stock_markets]
    tickers = pd.concat([pd.read_csv(tf) for tf in ticker_files], ignore_index=True)
    return tickers


stock_markets = ['NASDAQ']
tickers = read_tickers(stock_markets)
tickers

,ticker,company
0,AAIT,iShares MSCI All Country Asia Information Tech...
1,AAL,"American Airlines Group, Inc."
2,AAME,Atlantic American Corporation
3,AAOI,"Applied Optoelectronics, Inc."
4,AAON,"AAON, Inc."
...,...,...
2962,ZN,Zion Oil & Gas Inc
2963,ZNGA,Zynga Inc.
2964,ZSPH,"ZS Pharma, Inc."
2965,ZU,"zulily, inc."


#### Загружаем исторические данные за 2021 год для полученных тикеров

In [108]:
def read_historical_data(tickers):
    '''Returns dict {ticker : historical data for the ticker}'''
    data_for_ticker = {}
    for index, (ticker, name) in tickers.iterrows():
        try:
            data = pd.read_csv(f'{DATA_PATH}/{ticker}.csv')
            if len(data) > 100:
                data_for_ticker[ticker] = data
        except:
            pass
    return data_for_ticker


data_for_ticker = read_historical_data(tickers)

#### Пример данных для AAPL:

In [109]:
data_for_ticker['AAPL']

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2020-11-30,116.597423,120.584682,116.437929,118.670799,169410200,0.0,0
1,2020-12-01,120.624557,123.076720,119.627742,122.329109,127728200,0.0,0
2,2020-12-02,121.631329,122.977035,120.504931,122.687958,89004200,0.0,0
3,2020-12-03,123.126555,123.385729,121.820730,122.548409,78967600,0.0,0
4,2020-12-04,122.209495,122.468669,121.132933,121.860611,78260400,0.0,0
...,...,...,...,...,...,...,...,...
248,2021-11-23,161.119995,161.800003,159.059998,161.410004,96041900,0.0,0
249,2021-11-24,160.750000,162.139999,159.639999,161.940002,69463600,0.0,0
250,2021-11-26,159.570007,160.449997,156.360001,156.809998,76959800,0.0,0
251,2021-11-29,159.369995,161.190002,158.789993,160.240005,88748200,0.0,0


#### Выбираем 20 активов

In [110]:
assets = ['MDLZ', 'MSFT', 'NXPI', 'PCAR', 'INTC',
          'NVDA', 'ILMN', 'DXCM', 'ROST', 'LULU']

assets_data = {}
for asset in assets:
    assets_data[asset] = data_for_ticker[asset]

#### Оцениваем математические ожидания доходностей

In [111]:
import pandas as pd
import numpy as np

def add_logret(ticker_data: pd.DataFrame, by_column='Close') -> pd.DataFrame:
    '''Returns @ticker_data with calculated "logret" and "-logret" columns'''
    ticker_data = ticker_data.assign(logret=np.log(ticker_data[by_column]).diff())
    ticker_data['-logret'] = ticker_data['logret'].mul(-1)
    return ticker_data


def get_logret_mean_std(tickers, data_map, by_column='Close') -> pd.DataFrame:
    '''Returns @result pd.DataFrame such that @result.loc[ticker] == [logret_mean, logret_std]'''
    result = pd.DataFrame(data=[], columns=['ticker', 'logret_mean', 'logret_std'])
    result.set_index('ticker', inplace=True)

    for ticker in tickers:
        ticker_data = data_map[ticker]
        ticker_logret = np.log(ticker_data[by_column]).diff()
        result.loc[ticker] = [ticker_logret.mean(), ticker_logret.std()]
    
    return result
  

for ticker in assets_data.keys():
    assets_data[ticker] = add_logret(assets_data[ticker])

estims = get_logret_mean_std(assets, assets_data)
estims

,logret_mean,logret_std
ticker,,
MDLZ,0.000164,0.009635
MSFT,0.001743,0.012638
NXPI,0.001406,0.023336
PCAR,-0.000078,0.015266
INTC,0.000118,0.020613
NVDA,0.003541,0.025897
ILMN,0.000500,0.020907
DXCM,0.002243,0.022329
ROST,0.000076,0.018766


#### Находим матрицу выборочных ковариаций доходностей 

In [112]:
def get_covariation_matrix(tickers,                          
                           data_map=data_for_ticker,
                           by_column='logret'):
    columns = pd.DataFrame()
    for ticker in tickers:
        columns[ticker] = data_map[ticker][by_column][1:]
        
    matrix = columns.cov()
    return matrix

cov_matrix = get_covariation_matrix(assets, data_map=assets_data)
cov_matrix

,MDLZ,MSFT,NXPI,PCAR,INTC,NVDA,ILMN,DXCM,ROST,LULU
MDLZ,0.000093,0.000032,0.000031,0.000024,0.000026,0.000018,0.000022,0.000026,0.000058,0.000038
MSFT,0.000032,0.000160,0.000129,0.000025,0.000089,0.000184,0.000097,0.000112,0.000056,0.000091
NXPI,0.000031,0.000129,0.000545,0.000112,0.000250,0.000343,0.000170,0.000184,0.000152,0.000192
PCAR,0.000024,0.000025,0.000112,0.000233,0.000090,0.000058,0.000025,0.000008,0.000089,0.000033
INTC,0.000026,0.000089,0.000250,0.000090,0.000425,0.000188,0.000103,0.000102,0.000111,0.000103
NVDA,0.000018,0.000184,0.000343,0.000058,0.000188,0.000671,0.000160,0.000203,0.000075,0.000210
ILMN,0.000022,0.000097,0.000170,0.000025,0.000103,0.000160,0.000437,0.000202,0.000012,0.000133
DXCM,0.000026,0.000112,0.000184,0.000008,0.000102,0.000203,0.000202,0.000499,0.000068,0.000161
ROST,0.000058,0.000056,0.000152,0.000089,0.000111,0.000075,0.000012,0.000068,0.000352,0.000105
LULU,0.000038,0.000091,0.000192,0.000033,0.000103,0.000210,0.000133,0.000161,0.000105,0.000378


In [113]:
import plotly.express as px

fig = px.imshow(cov_matrix, title='Covariation Matrix',
                color_continuous_scale=px.colors.diverging.RdYlGn[::-1])
fig.show()

#### Проверяем вырожденность матрицы и число обусловленности

In [114]:
det = np.linalg.det(cov_matrix)
print(f'det = {det}')

con = np.linalg.cond(cov_matrix, p='fro')
print(f'con = {con}')

det = 1.1829872583873188e-36
con = 37.80303663511966


### 1. Истинный оптимальный портфель в модели Марковица с заданным отношением к риску. 

С заданным отношением к риску  подобираем константу b таким образом, что истинный оптимальный CVaR портфель совпадает с истинным оптимальным портфелем п.1. Значение константы смотри в упражнениях к теме.

$$ b = \frac{1}{\sqrt{2 \pi }} \frac{1}{(1 - \beta)} exp(-(\Phi^{-1}(\beta))^2 / 2)$$

In [115]:
import math
from scipy.stats import norm

beta = 0.90
b = (1 / math.sqrt(2 * math.pi)) * (1 / (1 - beta)) * np.exp(-(norm.ppf(beta)**2 / 2)) 
print(f'b = {b}')

b = 1.754983319324869


#### Решаем задачу оптимизации 

$$ -E(x)+ b \cdot σ(x) \rightarrow min $$
при условиях:
$$ x_1 + x_2 + ... +x_N = 1 $$
$$ x_i >= 0 $$
где
$$ E(x)= E_1x_1 + E_2 x_2 + ... + E_Nx_N $$
$$ σ^2(x)=\Sigma\Sigma σ_{i,j} x_i \cdot x_j $$

In [116]:
import math 
import numpy as np
from scipy.optimize import minimize

def get_E(x, vector_E):
    E = sum([(E_i * x_i) for E_i, x_i in zip(vector_E, x)]) 
    return E


def get_sigma(x, matrix_cov):
    sigma_squared = 0
    
    for i in range(len(x)):
        for j in range(len(x)):
            simga_i_j = matrix_cov.iloc[i].iloc[j]
            sigma_squared += simga_i_j * x[i] * x[j]
    
    sigma = math.sqrt(sigma_squared)
    return sigma
            
            
def target_function(x, vector_E, matrix_cov):
    E = get_E(x, vector_E)
    sigma = get_sigma(x, matrix_cov)
    result = -E + b * sigma
    return result


def find_optimal(vector_E=estims['logret_mean'], matrix_cov=cov_matrix):
    x0 = np.array([1/len(assets)] * len(assets))
    solution = minimize(target_function, x0, args=(vector_E, matrix_cov), 
                        method='SLSQP', 
                        constraints=[{'type': 'eq',  'fun': lambda x: sum(x) - 1}],
                        bounds=[(0, 1)] * len(assets))
    if not solution.success:
        raise Exception(opt.message)
    return solution
    
opt_solution = find_optimal()

#### Веса портфеля:

In [117]:
import plotly.express as px

print(f'Сумма весов: {sum(opt_solution.x)}')
fig = px.bar(x=assets, y=opt_solution.x)
fig.show()

Сумма весов: 1.0000000000000002


#### Значение целевой функции:

In [118]:
print(opt_solution.fun)

0.013285159466986446


### 2. Оценка неопределенности оптимального портфеля в модели Марковица с заданным отн. к риску. 

#### 2.1 Задаём число наблюдений T=30. С помощью генератора многомерного нормального распределения создаём выборку размера Т из нормального распределения с вектором математических ожиданий  E=(E1, E2, …, EN) и матрицей ковариаций (σi,j). 

In [119]:
T = 30
sample_raw = np.random.multivariate_normal(estims['logret_mean'], cov_matrix, T)
sample = pd.DataFrame(columns=assets, data=sample_raw)
sample

,MDLZ,MSFT,NXPI,PCAR,INTC,NVDA,ILMN,DXCM,ROST,LULU
0,-0.000242,0.006250,0.010342,0.022893,-0.003962,0.028379,0.005907,-0.001782,0.035798,0.015826
1,0.008993,-0.001309,0.006340,-0.010959,-0.004637,0.002350,0.008924,0.020711,-0.003499,-0.000765
2,0.009477,-0.020600,-0.029733,-0.006471,-0.001687,-0.044907,-0.026437,-0.008850,-0.012973,-0.013768
3,0.007108,0.005272,0.007680,0.012846,0.007970,0.011661,-0.020083,0.015534,0.023675,-0.000404
4,0.020143,-0.015587,-0.035515,-0.001424,-0.027856,-0.018718,-0.049860,-0.023759,-0.012257,-0.034198
5,-0.010992,-0.005254,0.000898,-0.005568,-0.017818,-0.017122,-0.028031,-0.006415,-0.003802,-0.000437
6,-0.009560,-0.012874,-0.032063,-0.035187,-0.014270,-0.004797,-0.003774,0.020112,-0.057573,-0.040830
7,0.005695,0.004634,0.006107,0.023258,-0.006936,0.029870,-0.010743,-0.008755,0.007343,-0.016589
8,-0.004556,0.005958,0.025552,-0.010110,0.021067,0.014827,0.015712,0.009172,0.001416,0.025198
9,0.007656,0.006681,0.011420,-0.023813,-0.023320,-0.000974,0.004244,0.010638,-0.008789,0.011805


#### 2.2 По построенной выборке делаем оценку Eest вектора математических ожиданий ...

In [120]:
estE = sample.mean()
estE

MDLZ    0.000943
MSFT    0.000795
NXPI   -0.000930
PCAR   -0.001556
INTC   -0.002846
NVDA    0.006921
ILMN   -0.004445
DXCM    0.004288
ROST   -0.000814
LULU   -0.000438
dtype: float64

#### ... и оценку (σesti,j) матрицы ковариаций. 

In [121]:
estCov = sample.cov()
estCov

,MDLZ,MSFT,NXPI,PCAR,INTC,NVDA,ILMN,DXCM,ROST,LULU
MDLZ,0.000098,0.000021,0.000012,0.000013,0.000005,0.000053,0.000006,0.000065,0.000075,0.000075
MSFT,0.000021,0.000111,0.000116,0.000063,0.000073,0.000161,0.000092,0.000097,0.000065,0.000132
NXPI,0.000012,0.000116,0.000474,0.000112,0.000138,0.000343,0.000141,0.000129,0.000253,0.000351
PCAR,0.000013,0.000063,0.000112,0.000282,0.000043,0.000141,-0.000032,0.000004,0.000148,0.000008
INTC,0.000005,0.000073,0.000138,0.000043,0.000221,0.000086,0.000128,0.000064,0.000107,0.000178
NVDA,0.000053,0.000161,0.000343,0.000141,0.000086,0.000699,0.000199,0.000349,0.000271,0.000388
ILMN,0.000006,0.000092,0.000141,-0.000032,0.000128,0.000199,0.000263,0.000168,0.000111,0.000233
DXCM,0.000065,0.000097,0.000129,0.000004,0.000064,0.000349,0.000168,0.000564,0.000146,0.000310
ROST,0.000075,0.000065,0.000253,0.000148,0.000107,0.000271,0.000111,0.000146,0.000443,0.000290
LULU,0.000075,0.000132,0.000351,0.000008,0.000178,0.000388,0.000233,0.000310,0.000290,0.000619


In [122]:
import plotly.express as px

fig = px.imshow(cov_matrix, title='Covariation Matrix',
                color_continuous_scale=px.colors.diverging.RdYlGn[::-1])
fig.show()

#### 2.3 Используя эти оценки решаем задачу оптимизации 

In [123]:
opt_solution = find_optimal(estE, estCov)

#### Находим и фиксируем веса портфеля ...

In [124]:
import plotly.express as px

true_opt = opt_solution.x
print(f'Сумма весов: {sum(opt_solution.x)}')
fig = px.bar(x=assets, y=opt_solution.x)
fig.show()

Сумма весов: 1.0


#### и значение целевой функции

In [125]:
print(opt_solution.fun)

0.01284281437143119


#### 2.4 Сравниаем два портфеля: истинный (п.1) и выборочный (п.2.3). Оцениваем относительную ошибку в определении весов портфеля в норме Manhattan (L1 норма Минковского). Делайте выводы и сравнение в системе координат (σ, E). 

In [126]:
from scipy.spatial.distance import cityblock
cityblock(true_opt, opt_solution.x)

0.0

#### 2.5 Повторите эксперимент S=40 раз и оцените среднюю относительную ошибку по S повторениям эксперимента. Сделайте выводы.  Сделайте сравнение в системе координат (σ, E). 

In [127]:
import copy

S = 40
errors = []
x_vectors = pd.DataFrame(columns = [f'x_{i}' for i in range(len(assets))], data=[])


for iteration in range(0, S):
    T = 30
    sample_raw = np.random.multivariate_normal(estims['logret_mean'], cov_matrix, T)
    sample = pd.DataFrame(columns=assets, data=sample_raw)
    
    estE = sample.mean()
    estCov = sample.cov()
    
    opt_solution = find_optimal(estE, estCov)
        
    errors.append(cityblock(true_opt, opt_solution.x))
    x_vectors.loc[iteration] = (copy.deepcopy(opt_solution.x))

    
print(f'Mean Erorr = {np.mean(errors)}')
print(f'Mean X:')
x_vectors.mean()

Mean Erorr = 0.7260271935017849
Mean X:


x_0    0.445824
x_1    0.164933
x_2    0.007146
x_3    0.150014
x_4    0.029795
x_5    0.024950
x_6    0.049528
x_7    0.031360
x_8    0.057276
x_9    0.039173
dtype: float64

In [128]:
import plotly.express as px

fig = px.bar(x=assets, y=x_vectors.mean(), title='Mean X')
fig.show()

#### 2.6  Предположите, что нам известны точные значения математических ожиданий E=(E1, E2, …, EN). Повторяем пп. 2.2-2.5. используя оценку только матрицы ковариаций

In [129]:
import copy

S = 40
errors = []
x_vectors = pd.DataFrame(columns = [f'x_{i}' for i in range(len(assets))], data=[])


for iteration in range(0, S):
    T = 30
    sample_raw = np.random.multivariate_normal(estims['logret_mean'], cov_matrix, T)
    sample = pd.DataFrame(columns=assets, data=sample_raw)
    
    estE = sample.mean()
    estCov = sample.cov()
    
    opt_solution = find_optimal(estims['logret_mean'], estCov)
        
    errors.append(cityblock(true_opt, opt_solution.x))
    x_vectors.loc[iteration] = (copy.deepcopy(opt_solution.x))

    
print(f'Mean Erorr = {np.mean(errors)}')
print(f'Mean X:')
x_vectors.mean()

Mean Erorr = 0.6604497286637576
Mean X:


x_0    0.482265
x_1    0.146272
x_2    0.006436
x_3    0.163107
x_4    0.025922
x_5    0.017903
x_6    0.050361
x_7    0.039567
x_8    0.037439
x_9    0.030728
dtype: float64

In [130]:
import plotly.express as px

fig = px.bar(x=assets, y=x_vectors.mean(), title='Mean X')
fig.show()